In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_csv('./dataset_ES.csv')

In [3]:
df1.head()

,tweets,label
0,10 error que el mujer cometer en el primero cita,1
1,increíble mascarilla borrar el mancha como si ...,1
2,5 ejercicio CON silla fácil práctico rápido de...,1
3,Huevo Hervido así él preparar este Excelente R...,1
4,el tipo más rallado en este mundo que tal tú p...,1


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.manifold import TSNE

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Conv1D, MaxPooling1D, Dropout
# from keras.utils.np_utils import to_categorical
# from keras.utils.np_utils import to_categorical

from tensorflow.keras.utils import to_categorical

In [5]:
tweets = df1['tweets'].astype(str)

In [6]:
label = df1['label'].astype(str)

In [7]:
train_text, test_text, train_y, test_y = train_test_split(tweets, label,test_size = 0.2)

In [8]:
MAX_NB_WORDS = 20000

# get the raw text data
texts_train = train_text.astype(str)
texts_test = test_text.astype(str)

# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS, char_level=False)
tokenizer.fit_on_texts(texts_train)
sequences = tokenizer.texts_to_sequences(texts_train)
sequences_test = tokenizer.texts_to_sequences(texts_test)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

c:\Users\choc-\OneDrive\Documents\Python\CIC\cic_redes\lib\site-packages\keras\src\preprocessing\text.py:246: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn(


Found 31852 unique tokens.


In [9]:
MAX_SEQUENCE_LENGTH = 150

# pad sequences with 0s
x_train = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
x_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', x_train.shape)
print('Shape of data test tensor:', x_test.shape)

Shape of data tensor: (24000, 150)
Shape of data test tensor: (6000, 150)


In [10]:
y_train = train_y
y_test = test_y

y_train = to_categorical(np.asarray(y_train))
print('Shape of label tensor:', y_train.shape)

Shape of label tensor: (24000, 2)


In [11]:
from keras.layers import Dense, Input, Flatten
from keras.layers import GlobalAveragePooling1D, Embedding
from keras.models import Model

EMBEDDING_DIM = 50
N_CLASSES = 2

# input: a sequence of MAX_SEQUENCE_LENGTH integers
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')

embedding_layer = Embedding(MAX_NB_WORDS, EMBEDDING_DIM,
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)
embedded_sequences = embedding_layer(sequence_input)

average = GlobalAveragePooling1D()(embedded_sequences)
predictions = Dense(N_CLASSES, activation='softmax')(average)

model = Model(sequence_input, predictions)
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['acc'])

In [12]:
model.fit(x_train, y_train, validation_split=0.1,
          epochs=40, batch_size=128)

Epoch 1/40


169/169 [==============================] - 3s 15ms/step - loss: 0.6891 - acc: 0.5701 - val_loss: 0.6831 - val_acc: 0.5858
Epoch 2/40
169/169 [==============================] - 2s 14ms/step - loss: 0.6708 - acc: 0.6297 - val_loss: 0.6565 - val_acc: 0.6450
Epoch 3/40
169/169 [==============================] - 2s 14ms/step - loss: 0.6435 - acc: 0.6487 - val_loss: 0.6334 - val_acc: 0.6521
Epoch 4/40
169/169 [==============================] - 2s 13ms/step - loss: 0.6169 - acc: 0.6743 - val_loss: 0.6161 - val_acc: 0.6754
Epoch 5/40
169/169 [==============================] - 2s 14ms/step - loss: 0.5903 - acc: 0.6994 - val_loss: 0.5994 - val_acc: 0.6654
Epoch 6/40
169/169 [==============================] - 2s 14ms/step - loss: 0.5656 - acc: 0.7207 - val_loss: 0.5834 - val_acc: 0.6929
Epoch 7/40
169/169 [==============================] - 2s 13ms/step - loss: 0.5406 - acc: 0.7478 - val_loss: 0.5715 - val_acc: 0.7021
Epoch 8/40
169/169 [==============================] - 2s 13ms/step 

In [13]:
output_test = model.predict(x_test)
print("test auc:", roc_auc_score(y_test,output_test[:,1]))

188/188 [==============================] - 0s 1ms/step
test auc: 0.7566424396123672


In [14]:
predicted_classes = np.argmax(output_test, axis=1)

In [15]:
df2 = pd.read_csv('./Test-ES-nolabels.csv')

In [16]:
pruebas1 = df2['tweet'].astype(str)

In [17]:
pruebas1.shape

(20000,)

In [18]:
pruebas = tokenizer.texts_to_sequences(pruebas1)
pruebas_finales = pad_sequences(pruebas, maxlen=MAX_SEQUENCE_LENGTH)


In [19]:
output_test1 = model.predict(pruebas_finales)

625/625 [==============================] - 1s 1ms/step


In [20]:
output_test1.shape

(20000, 2)

In [21]:
finales = np.argmax(output_test1, axis=1)

In [22]:
finales.shape

(20000,)

In [23]:
lista_finales = finales.tolist()

In [24]:
df2['labels_predichos'] = lista_finales

In [25]:
df2.to_csv('Test-ES-labels.csv')